In [1]:
!pip install -q wandb
import wandb
import pathlib
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, Model
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from PIL import Image

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
################################################################
# Preparing training (without augmentation) and validation set 
################################################################
# Preparing training and validation sets without augmentation
# Loading data from directory
# data_dir = pathlib.Path('/content/drive/MyDrive/inaturalist_12K/train') # Set path to the right directory
data_dir = '/content/drive/MyDrive/inaturalist_12K/train' # Set path to the right directory
train_data = tf.keras.preprocessing.image_dataset_from_directory(
                      directory = data_dir,
                      labels = 'inferred',  
                      label_mode = 'categorical',
                      color_mode = 'rgb',
                      batch_size = 32,
                      image_size = (256, 256),
                      shuffle = True,
                      seed = 17,
                      validation_split = 0.2,
                      subset = 'training')

val_data = tf.keras.preprocessing.image_dataset_from_directory(
                      directory = data_dir,
                      labels = 'inferred',  
                      label_mode = 'categorical',
                      color_mode = 'rgb',
                      batch_size = 32,
                      image_size = (256, 256),
                      shuffle = True,
                      seed = 17,
                      validation_split = 0.2,
                      subset = 'validation')

# Retaining 25 percent of train and validation data and discarding the rest
len_train, len_val = len(train_data), len(val_data)
train_data = train_data.take(int(0.25*len_train))
val_data = val_data.take(int(0.25*len_val))

Found 10000 files belonging to 10 classes.
Using 8000 files for training.
Found 10000 files belonging to 10 classes.
Using 2000 files for validation.


In [4]:
################################################################
# Preparing training set with augmentation 
################################################################
train_data_augmenter = ImageDataGenerator(
                            rescale = None,
                            rotation_range = 20,
                            width_shift_range = 0.2,
                            height_shift_range = 0.2,
                            brightness_range = [0.2, 1.5],
                            shear_range = 0.2,
                            zoom_range = 0.2,
                            horizontal_flip=True,
                            data_format = 'channels_last',
                            validation_split = 0.2)        #Specifying parameters for augmentation of training data

val_data_augmenter = ImageDataGenerator(validation_split = 0.2) #No augmentation of validation data

train_aug_gen = train_data_augmenter.flow_from_directory(data_dir, shuffle = True, \
                                                         seed = 17, subset = 'training')
val_aug_gen = val_data_augmenter.flow_from_directory(data_dir, shuffle = True, \
                                                     seed = 17, subset = 'validation')

train_aug_data = tf.data.Dataset.from_generator(
                    lambda: train_aug_gen,
                    output_types = (tf.float32, tf.float32),
                    output_shapes = ([None, 256, 256, 3], [None, 10]))

val_aug_data = tf.data.Dataset.from_generator(
                  lambda: val_aug_gen,
                  output_types = (tf.float32, tf.float32),
                  output_shapes = ([None, 256, 256, 3], [None, 10]))

# Retaining 25 percent of train and validation data and discarding the rest
train_aug_data = train_aug_data.take(int(0.25*len_train))
val_aug_data = val_aug_data.take(int(0.25*len_val))

Found 8000 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.


Using tutorials:
- https://www.tensorflow.org/tutorials/text/image_captioning
- https://www.tensorflow.org/tutorials/images/transfer_learning


In [5]:
class CNN(Model):
    def __init__(self, base_model_name, tune=False, offset=20):
        super(CNN, self).__init__()
        self.IMG_SHAPE = (256, 256, 3)
        self.base_model_name = base_model_name
        self.init_base_model()
        
        if tune:
            self.base_model.trainable = True
            fine_tune_at = len(self.base_model.layers)-offset
            # Freeze all the layers before the `fine_tune_at` layer
            for layer in self.base_model.layers[:fine_tune_at]:
                layer.trainable =  False
        else:
            self.base_model.trainable = False

        self.additional_avg_pool1 = layers.GlobalAveragePooling2D()
        self.additional_dense1 = layers.Dense(10)

    def init_base_model(self):
        if self.base_model_name == "InceptionV3":
            self.base_model = tf.keras.applications.InceptionV3(input_shape=self.IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')
        elif self.base_model_name == "InceptionResNetV2":
            self.base_model = tf.keras.applications.InceptionResNetV2(input_shape=self.IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')
        elif self.base_model_name == "ResNet50":
            self.base_model = tf.keras.applications.ResNet50(input_shape=self.IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')
        elif self.base_model_name == "Xception":
            self.base_model = tf.keras.applications.Xception(input_shape=self.IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')
            
    def call(self, inputs):
        x = self.base_model(inputs)
        x = self.additional_avg_pool1(x)
        return self.additional_dense1(x)

In [6]:
###############################################
# Listing the hyperparameters in wandb config 
###############################################
sweep_config = {'name': 'random-test-sweep', 'method': 'grid'}
sweep_config['metric'] = {'name': 'val_acc', 'goal': 'maximize'}
parameters_dict = {
                   'base_model_name': {'values': ["InceptionV3", "InceptionResNetV2", "ResNet50", "Xception"]},
                   'tune': {'values': [False, True]},
                  }
sweep_config['parameters'] = parameters_dict

In [7]:
def pretrain_CNN_sweep(config=sweep_config):
    with wandb.init(config=config):
        config = wandb.init().config
        wandb.run.name = 'BM_{}_tune_{}'.format(config.base_model_name, \
                                                          config.tune)
        
        model = CNN(config.base_model_name, config.tune)
        base_learning_rate = 0.0001
        model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
                loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
        
        history = model.fit(train_data, epochs=10, validation_data=val_data, \
                            callbacks = [wandb.keras.WandbCallback()])

In [ ]:
#################################
# Setting up wandb sweeps
#################################
sweep_id = wandb.sweep(sweep_config, project = 'DL-Assignment2-PartB-9April')
wandb.agent(sweep_id, function=pretrain_CNN_sweep)

Create sweep with ID: rym5zond
Sweep URL: https://wandb.ai/sowmyamanojna/DL-Assignment2-PartB-9April/sweeps/rym5zond


wandb: Agent Starting Run: ydnoo4aj with config:
wandb: 	base_model_name: InceptionV3
wandb: 	tune: False
wandb: Currently logged in as: sowmyamanojna (use `wandb login --relogin` to force relogin)


Epoch 1/10
62/62 [==============================] - 54s 701ms/step - loss: 18.0064 - accuracy: 0.0855 - val_loss: 9.9351 - val_accuracy: 0.0875


wandb: ERROR Can't save model, h5py returned error: Saving the model to HDF5 format requires the model to be a Functional model or a Sequential model. It does not work for subclassed models, because such models are defined via the body of a Python method, which isn't safely serializable. Consider saving to the Tensorflow SavedModel format (by setting save_format="tf") or using `save_weights`.


Epoch 2/10
62/62 [==============================] - 43s 664ms/step - loss: 9.4341 - accuracy: 0.1057 - val_loss: 8.6453 - val_accuracy: 0.1167
Epoch 3/10
62/62 [==============================] - 44s 664ms/step - loss: 8.2158 - accuracy: 0.1288 - val_loss: 7.8264 - val_accuracy: 0.1187
Epoch 4/10
62/62 [==============================] - 44s 664ms/step - loss: 7.3945 - accuracy: 0.1325 - val_loss: 7.2020 - val_accuracy: 0.1250
Epoch 5/10
62/62 [==============================] - 44s 670ms/step - loss: 6.7160 - accuracy: 0.1509 - val_loss: 7.0768 - val_accuracy: 0.1208
Epoch 6/10
62/62 [==============================] - 44s 664ms/step - loss: 6.2827 - accuracy: 0.1445 - val_loss: 6.3817 - val_accuracy: 0.1146
Epoch 7/10
62/62 [==============================] - 44s 664ms/step - loss: 5.9522 - accuracy: 0.1647 - val_loss: 6.3229 - val_accuracy: 0.1208
Epoch 8/10
62/62 [==============================] - 44s 667ms/step - loss: 5.7501 - accuracy: 0.1724 - val_loss: 6.1239 - val_accuracy: 0.1333